In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

In [2]:
def data_generator(r, l, e, tam=1000, rho=False):
    '''
        Entrada:
            r, l, e: Numeros Naturais maiores que zero
            tam : Valor maximo de cada elemento, padrao: 1000.
            rho : Flag para retornar apenas uma matriz l x r
        Saida:
            vetor: tamanho r, matriz: ordem l x r, matriz: ordem e x r
        Erro:
            tupla: None, print do erro.
    '''
    less_than = l <= (r - 1) and e <= (r - 1)
    greather_than_zero = l > 0 and e > 0 and r > 0

    if not (less_than and greather_than_zero):
        print("Data Generator Error: Bad Entry.")
        return (None, None, None)
    else:
        if rho:
            matriz_lr = np.random.randint(0, tam + 1, size=(l, r))
            return (matriz_lr)
        else:
            vector = np.random.randint(0, tam + 1, size=r)
            matriz_lr = np.random.randint(0, tam + 1, size=(l, r))
            matriz_er = np.random.randint(0, tam + 1, size=(e, r))

        return (vector, matriz_lr, matriz_er)

# ---------------------------------------------------------------------------------------------------

def generator_conditional(r, l, e, tam=1000):
    """
        Entradas:
            r, l, e: Numeros naturais maiores que zero
            tam = tamanho maximo para numero aleatorio
            t = variação maxima do gerador do conjunto Q
        Saidas:
            c: Vetor de tamanho r
            rho: Matriz de ordem l x r
            pi: Matriz de ordem e x r
            rho_r: vetor de tamanho l
            pi_r: vetor de tamanho e
            Q: Conjunto de tamanho r + 1 tal que U0 = Ur
    """

    c, rho, pi = data_generator(r, l, e, tam=tam)

    # Ultima coluna das matrizes Rho e Pi respectivamente
    rho_r = rho[:, -1:]
    pi_r = pi[:, -1:]

    return (c, rho, pi, rho_r, pi_r)

In [3]:
def to_list(vet):
    lista = []
    for x in vet:
        lista.append(x[0])

    return np.array(lista)

In [4]:
def prepara_dual(c, rho, pi, rho_r, pi_r):
    funcao_objetivo = np.concatenate((rho_r, pi_r))
    restricao = np.concatenate((rho.T, pi.T), axis=1)
    funcao_objetivo = to_list(funcao_objetivo)
    funcao_objetivo = np.dot(-1, funcao_objetivo)
    
    return(funcao_objetivo, restricao, c)

In [55]:
c, rho, pi, rho_r, pi_r = generator_conditional(4, 3, 3)
objetivo, restricao, vetor = prepara_dual(c, rho, pi, rho_r, pi_r)
bound = [(None, None),]* rho_r.size + [(0, None), ] * pi_r.size

In [56]:
primal = linprog(c=c, A_ub=pi, b_ub=pi_r, A_eq=rho, b_eq=rho_r)
dual = linprog(c=objetivo, A_ub=restricao, b_ub=vetor, bounds=bound)

In [57]:
primal

     fun: 275.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 0., 1.])

In [58]:
dual

     fun: -275.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([184.32719836, 589.03476483, 767.79141104,   0.        ])
  status: 0
 success: True
       x: array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.28118609])

In [39]:
primal_optimo = np.dot(c, primal.x)
dual_optimo = -1 * np.dot(objetivo, dual.x)

In [40]:
print("Primal: ", primal_optimo)
print("Dual: ", dual_optimo)

Primal:  945.0
Dual:  942.8331734052058


In [42]:
if primal_optimo == dual_optimo:
    print("Igual")
elif primal_optimo <= dual_optimo + 10 or primal_optimo >= dual_optimo - 10:
    print("Dentro da faixa")
else:
    print("Diferente")

Dentro da faixa


In [62]:
def dados_tabelar(r, maximo):
    n = 1
    primais = []
    duais = []
    status = []
    erres = []
    it_primal = []
    it_dual = []
    
    while True:
        # Prepara os dados
        c, rho, pi, rho_r, pi_r = generator_conditional(r, r-1, r-1)
        objetivo, restricao, vetor = prepara_dual(c, rho, pi, rho_r, pi_r)
        bound = [(None, None),]* rho_r.size + [(0, None), ] * pi_r.size
        
        # Executa os Simplexes
        primal = linprog(c=c, A_ub=pi, b_ub=pi_r, A_eq=rho, b_eq=rho_r)
        dual = linprog(c=objetivo, A_ub=restricao, b_ub=vetor, bounds=bound)
        
        # Calcula o valor optimo
        primal_optimo = np.dot(c, primal.x)
        dual_optimo = -1 * np.dot(objetivo, dual.x)
        
            
        # Salva os dados
        primais.append(primal.fun)
        duais.append(-1 * dual.fun)
        erres.append(r)
        it_primal.append(primal.nit)
        it_dual.append(dual.nit)
        
        # Iteracao
        n += 1
        
        if (n % 20) == 0:
            r += 5
        if r == maximo:
            break
    
    # Cria data frame com os dados salvos
    df = pd.DataFrame()
    df["R"]                    = erres
    df["Optimo Primal"]        = primais
    df["Optimo Dual"]          = duais
    df["Iteracao Primal"]      = it_primal
    df["Iteracao Dual"]        = it_dual
    
    return (df)

In [63]:
dados = dados_tabelar(5, 30)

In [64]:
dados.to_excel("teste.xlsx")